<a href="https://colab.research.google.com/github/mojoblue/Kaggle_BBC_news_classification/blob/master/BBC_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from gensim.parsing.preprocessing import preprocess_string
import pandas as pd
import io

Mount drive files

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Do preprocessing news data text using gensim
https://radimrehurek.com/gensim/parsing/preprocessing.html

In [0]:
category2Index = {
    "sport" : 0,
    "tech" : 1,
    "business" : 2,
    "entertainment" : 3,
    "politics" : 4
}

In [0]:
index2Category = {
    0 : "sport",
    1 : "tech",
    2 : "business",
    3 : "entertainment",
    4 : "politics"
}

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/learn-ai-bbc/BBC News Train.csv', encoding="utf-8")

In [0]:
# preprocessed news data( Text => preprocessed Word ) and category2index
df['Text'] = df['Text'].map(lambda x : preprocess_string(x))
df['Category'] = df['Category'].map(lambda x: category2Index[x])

In [7]:
print(df['Text'][:2])

0    [worldcom, boss, launch, defenc, lawyer, defen...
1    [german, busi, confid, slide, german, busi, co...
Name: Text, dtype: object


In [8]:
print(df['Category'][:2])

0    2
1    2
Name: Category, dtype: int64


Load GloVe model twitter 100 dimension


In [0]:
def loadGloveModel(gloveFile):
  print("Loading GloVe Model")
  model = {}
  with open(gloveFile) as f:
    for line in f:
      splitLine = line.split()
      word = splitLine[0]
      embedding = np.array([float(val) for val in splitLine[1:]])
      model[word] = embedding
  print("Done.", len(model), " words loaded!")
  return model

In [10]:
gloveModel = loadGloveModel('/content/gdrive/My Drive/learn-ai-bbc/glove.twitter.27B.100d.txt')

Loading GloVe Model
Done. 1193515  words loaded!


Make baseline model using BOW

In [0]:
def makeBOW(dataframe):
  BOW_dict=dict()
  BOW_word_dict = dict()
  for wordList in df['Text']:
    for word in wordList:
      BOW_dict.setdefault(word, 0)
      BOW_dict[word] += 1
  for i, item in enumerate(BOW_dict.keys()):
    BOW_word_dict[item] = i
  
  BOW_all_sentences_list=[]
  for wordList in df['Text']:
    BOW_sentence_list = [0 for i in range(len(BOW_dict.keys()))]
    for word in wordList:
      BOW_sentence_list[BOW_word_dict[item]] += 1
    BOW_all_sentences_list.append(BOW_sentence_list)
  return BOW_all_sentences_list

In [0]:
BOW_list = makeBOW(df)

In [41]:
print(BOW_list[:3])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,